In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import files

def upload_image():
    uploaded = files.upload()
    if uploaded:
        file_name = next(iter(uploaded))
        image = cv2.imread(file_name, cv2.IMREAD_GRAYSCALE)
        return image
    else:
        print("No files were uploaded.")
        return None

def kernel(image, kernel):
    k_h, k_w = kernel.shape
    pad_h, pad_w = k_h // 2, k_w // 2
    padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
    output = np.zeros_like(image)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            output[i, j] = np.sum(padded_image[i:i+k_h, j:j+k_w] * kernel)

    return output

def gaussian_blur(image, kernel_size=7, sigma=1.0):
    k = kernel_size // 2
    x = np.arange(-k, k+1)
    y = np.arange(-k, k+1)
    X, Y = np.meshgrid(x, y)
    gauss_kernel = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
    gauss_kernel /= gauss_kernel.sum()
    return kernel(image, gauss_kernel)

def laplacian(image):
    blurred = gaussian_blur(image, kernel_size=7)
    lap = kernel(blurred, np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=np.float32))
    results = np.clip(blurred - lap, 0, 255).astype(np.uint8)
    return results

def high_boost(image, alpha=1.5):
    blurred = gaussian_blur(image, kernel_size=7)
    high_boost_img = np.clip(blurred + alpha * (image - blurred), 0, 255).astype(np.uint8)
    return high_boost_img

def unsharp(image, blur_size=(7, 7), alpha=1.5):
    blurred = gaussian_blur(image, kernel_size=blur_size[0])
    mask = image - blurred
    sharpened = np.clip(image + alpha * mask, 0, 255).astype(np.uint8)
    return sharpened

def compute_derivatives(image):
    rows, cols = image.shape

    dx_image = np.zeros((rows, cols))
    dy_image = np.zeros((rows, cols))
    dxx_image = np.zeros((rows, cols))
    dyy_image = np.zeros((rows, cols))
    dxy_image = np.zeros((rows, cols))
    laplacian_image = np.zeros((rows, cols))

    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                dx_image[i, j] = image[i, j + 1] - image[i, j]
            if i < rows - 1:
                dy_image[i, j] = image[i + 1, j] - image[i, j]
            if i < rows - 1 and j < cols - 1:
                dxy_image[i, j] = dx_image[i, j] + dy_image[i, j]

    for i in range(rows):
        for j in range(cols):
            if j > 0 and j < cols - 1:
                dxx_image[i, j] = dx_image[i, j] - dx_image[i, j - 1]
            if i > 0 and i < rows - 1:
                dyy_image[i, j] = dy_image[i, j] - dy_image[i - 1, j]
            if i > 0 and i < rows - 1 and j > 0 and j < cols - 1:
                laplacian_image[i, j] = dxx_image[i, j] + dyy_image[i, j]

    plt.figure(figsize=(12, 8))

    plt.subplot(2, 3, 1)
    plt.title("First Derivative in X")
    plt.imshow(dx_image, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 3, 2)
    plt.title("First Derivative in Y")
    plt.imshow(dy_image, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 3, 3)
    plt.title("First Derivative in XY")
    plt.imshow(dxy_image, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 3, 4)
    plt.title("Second Derivative in X")
    plt.imshow(dxx_image, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 3, 5)
    plt.title("Second Derivative in Y")
    plt.imshow(dyy_image, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 3, 6)
    plt.title("Laplacian (Second Derivative in XY)")
    plt.imshow(laplacian_image, cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# Usage
image = upload_image()
if image is not None:
    compute_derivatives(image)
    laplacian_image = laplacian(image)
    high_boost_image = high_boost(image)
    unsharp_image = unsharp(image)

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.title("Laplacian Sharpening")
    plt.imshow(laplacian_image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("High-Boost Filtering")
    plt.imshow(high_boost_image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Unsharp Masking")
    plt.imshow(unsharp_image, cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()
